# Nearest Neighbour & KNN
in diesem Notebook betrachten wir die einfachen Klassifikationsalgorithmen Nearest Neigbour & K-Nearest Neigbour. 

Vorbedingung ist, dass gelablete Daten vorliegen und eine Klassifzierung gewünscht ist




## Nearest Neigbour NN
Am Beispiel von Bild Klassifikation werden hier einfach Test bilder gegen alle Bilder der Trainbild verglichen. "Vergleich" bedeutet in diesem Falle, dass die Bilder übereinander gelegt werden, und die Pixelwerte voneinander abgzogen werden. Anschließend kommuliet man jeden Wert zwischen dem einen Testbild und alle Trainbildern zunächst individuell auf, und legt den aufkummulierten Wert in einer List ab. Somit enthält die Liste alle Summen der Differenzen zwischen dem einzelnen Test-Bild und allen Trainbildern. Wir gehen nun davon aus, dass jenes gelablte TrainingsBild, welches die geringste aufkummmuliet Differenz zu dem ungelableten Testimage besitzt, ddem Testbild am ähnlichsten ist. somit labeln wir dem Testimage, das entsprechende Label. Vereinfacht gesprochen werden bei der NN lediglich die beiden Instanzen miteinander verglichen. Für die Distanzmessung wird entweder die Euklidische oder die Manhatten Distanz verwendet

## KNearest Neigbour KNN
KNN ist die Erweiterung zu NN. In diesem Beispiel wird jedoch nicht ausschließlcih der nächste Nachbar, sondern die "k"-näcshten Nachbarn für die Zuordnung gewählt

## Vorgehen in diesem Notebook
zunächst werden ich die beiden Algorithmen zu NN und KNN selbst implementieren. Anschließend verwende ich die SK-Learn Bibliothek.

In [1]:
import numpy as np

In [2]:
# Laden der Daten, diese wurden bereits im vorherigen Notebook gelden, entapckt, normalisiert & vektorisiert habe
train_X = np.load('./DATA/train_X.npy')
train_y = np.load('./DATA/train_y.npy')
test_X = np.load('./DATA/test_X.npy')
test_y = np.load('./DATA/test_y.npy')

In [3]:
print((train_X.shape))
print((train_y.shape))
print((test_X.shape))
print((test_y.shape))

(50000, 3072)
(50000, 10)
(10000, 3072)
(10000, 10)


In [4]:
CIFAR_10_CLASSES = ["Flugz.","PKW","Vogel","Katze","Wild","Hund", "Frosch", "Pferd", "Schiff","LKW"]

## Eigene Erstellung des NN-Algorithmus mit Erweiterung zu KNN

In [ ]:
class NearestNeighbor(object):
    def __init__(self):
        pass

    def train(self, X, y):
        self.X = X
        self.y = y

    def predictSinglePicture(self, test_X_single, k=1):
        distances = []
        for i in range(0, len(self.X), 1):
            currentDiff = self.X[i] - test_X_single
            distances.append(np.argmin(currentDiff))
        indexOfPictureWithMindDif = np.argmin(distances)
        print(train_y[indexOfPictureWithMindDif])
        

    def predictTestSet(self, test_X, test_y, train_y):
        correctClassified = 0
        falesClassified = 0
        for i in range(0, len(test_X), 1): #wdh 1000x
            realPicture = test_y[i]
            distances = []
            #for j in range(0, len(self.X),1): # wdh 50000x
            for j in range(0, 500,1): # wdh 50000x # for testing smaller set, just the first 500 not 50000 of the train set
                currentDiff = self.X[j] - test_X[i]
                distances.append(np.argmin(currentDiff))
            indexOfPictureWithMindDif = np.argmin(distances)

            if(str(train_y[indexOfPictureWithMindDif]) == str(realPicture)):
                correctClassified = correctClassified + 1
            else:
                falesClassified = falesClassified + 1    
            if(i % 100 == 0):
                print(i)
                print(correctClassified)
                print(falesClassified)
                print(correctClassified/(correctClassified+falesClassified))
                print(falesClassified/(correctClassified+falesClassified))
                print()
        print("Final:")    
        print(correctClassified)
        print(falesClassified)
        

In [ ]:
NN = NearestNeighbor()
NN.train(train_X, train_y)
NN.predictTestSet(test_X, test_y, train_y) # correct13 / false87

## Bsp-Prediction eines einzelnen Wertes
im folgenden Block kann mittels der Variable num, ein Bild aus dem Testset gewählt werden. Dieses wird anschließend gegen alle Bilder des Train-Sets verglichen. Jenes bild, welches am wenigsten Differenz, aufweist, wird retour geliefert.

In [ ]:
#BSP-Predicitio für bild "num"
num = 13
NN.predictSinglePicture(test_X[num])
print(test_y[num])

## Verwendung der KNN-Methode von SK-Learn
Diese Methode erwartet als Input einen 2-Dimensionalen Vector, daher muss zunächst ein Reshaping stattfinden(was bereits im ersten Notebook stattfand). Die obenstehenden Implementierungen sind insofern "aufwendig". mit den folgenden Zeilen konnte ich jedoch gut meine eigene Arbeit gegenprüfen 

# KNN -selbsterstellt
Knn mit Soft-Voting. Das heist von den nächsten Nachbarn werden zuletzt die Distanzen der gleichen Klassen aufsummiert und durch die anzahl der Klassenteilnehmer geteilt.

In [ ]:
class KNearestNeighbor(object):
    def __init__(self):
        pass

    def train(self, X, y, labels):
        self.X = X
        self.y = y
        self.labels = labels
        
    def predictTestSet(self, test_X, test_y, train_y, k=1):
        correctClassified = 0
        falesClassified = 0
        kNeigbours = []
        for i in range(0, len(test_X), 1): #wdh 1000x
            realPicture = test_y[i]
            distances = []
            #for j in range(0, len(self.X),1): # wdh 50000x
            for j in range(0, 200,1): # wdh 50000x # for testing smaller set, just the first 500 not 50000 of the train set
                currentDiff = self.X[j] - test_X[i]
                distances.append([np.argmin(currentDiff), j])

            sortedTopK = (sorted(distances, key=lambda s:s[0])[:k+1])
            k_Classes = []
            checklist1 = [] # prüfung ob wert in der list
            checklist2 = []
            for k in range(0, len(sortedTopK),1):
                searchlist = list(self.y[sortedTopK[k][1]])
                categorie = (self.labels[searchlist.index(1)])
                dif = (sortedTopK[k][0])
                # print(categorie)
                # print(sortedTopK[k][1])
                if(categorie in checklist1):
                    idx = (checklist1.index(categorie))
                    checklist2[idx][1] = checklist2[idx][1] + sortedTopK[k][0]
                    checklist2[idx][2] = checklist2[idx][2] + 1
                else:
                    checklist1.append(categorie)
                    checklist2.append([categorie, sortedTopK[k][0], 1])
            evalList = []
            for element in range(0, len(checklist2),1):
                evalList.append([checklist2[element][1]/checklist2[element][2], checklist2[element][0]])
                
            sortEvallist = (sorted(evalList, key=lambda s:s[0])[:k+1])
            pred_y = sortEvallist[0][1]
            
            # Prüfen ob realität oder nicht         
            #print(pred_y)
            #print(self.labels[list(realPicture).index(1)])
            if(pred_y == self.labels[list(realPicture).index(1)]):
                correctClassified = correctClassified + 1
            else:
                falesClassified = falesClassified + 1
                
        print(correctClassified)
        print(falesClassified)
        print(correctClassified/(correctClassified+falesClassified))
        print(falesClassified/(correctClassified+falesClassified))
        print()
                                              
                                              
               
                
            
           
            
        

In [ ]:
KNN = KNearestNeighbor()
KNN.train(train_X, train_y, CIFAR_10_CLASSES)
#KNN.predictTestSet(test_X, test_y, train_y, 5, CIFAR_10_CLASSES) # correct13 / false87
k = 2
KNN.predictTestSet(test_X, test_y, train_y, k) # correct13 / false87

# NN ist ein Spezialfall von KNN
wie oben 

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
k_range = [1,3,5,7] # bie k == 1 habe ich NN, alle weiteren sind 
scores = {}
scores_list = []
train_X = train_X[:500]
train_y = train_y[:500]
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(train_X, train_y)
    pred_y = knn.predict(test_X)
    scores[k] = metrics.accuracy_score(test_y,pred_y)
    scores_list.append(metrics.accuracy_score(test_y, pred_y))
    print(scores_list)
print(scores_list)

[0.2186]
[0.2186, 0.1595]
[0.2186, 0.1595, 0.1305]
[0.2186, 0.1595, 0.1305, 0.0971]
[0.2186, 0.1595, 0.1305, 0.0971]
